# ETL de BegInvFINAL12312016

## Carga de librerías y dataset

In [1]:
# Cargamos las librerías necesarias
import pandas as pd
import numpy as np

In [2]:
# Cargamos el dataset csv
df = pd.read_csv(r"C:\Users\USUARIO\Downloads\archive\BegInvFINAL12312016.csv")

# Convención de nomenclatura de los nombres de las columnas

In [3]:
# Convertir los monbres de las columnas a snake_case

def to_snake_case(col_name):
    """
    Convierte nombres de columna a snake_case insertando un '_' antes de cada mayúscula
    y convirtiendo todo a minúsculas.
    """
    # Usaremos una comprensión de listas para construir el nuevo nombre
    new_name = ''
    for char in col_name:
        if char.isupper():
            # Si el caracter es mayúscula, añade un guion bajo ANTES y luego la minúscula.
            # Evita añadir un guion bajo al principio (si no está vacío).
            if new_name and not new_name.endswith('_'):
                new_name += '_'
            new_name += char.lower()
        else:
            # Si es minúscula o número, solo añade el carácter
            new_name += char
            
    # Limpia cualquier caracter especial (espacios, etc.) por guiones bajos, 
    # y luego elimina guiones dobles y los del inicio/final.
    return new_name.replace(' ', '_').strip('_')


# Aplicar la función a todas las columnas del DataFrame
df.columns = [to_snake_case(col) for col in df.columns]

# --- VERIFICACIÓN ---
print("--- Nombres de Columna Después de la Transformación (Snake Case Simple) ---")
print(df.columns.tolist())

--- Nombres de Columna Después de la Transformación (Snake Case Simple) ---
['inventory_id', 'store', 'city', 'brand', 'description', 'size', 'on_hand', 'price', 'start_date']


##  Tratamiento de la columna size

In [4]:
# Exploramos la columna size
print("Valores únicos y sus conteos más frecuentes (Top 50):")
print(df['size'].value_counts().head(50))

print("\nValores únicos y sus conteos menos frecuentes (Bottom 50):")
print(df['size'].value_counts().tail(50))

Valores únicos y sus conteos más frecuentes (Top 50):
size
750mL         144514
1.75L          21774
1.5L           12273
50mL            9460
375mL           7413
Liter           3671
3L              2386
5L              1534
187mL 4 Pk       527
500mL            425
4L               328
50mL 4 Pk        298
187mL            275
50mL 5 Pk        170
200mL            162
300mL            148
100mL            139
750mL 2 Pk       122
50mL 3 Pk        105
100mL 4 Pk        96
200mL 4 Pk        93
250mL 4 Pk        74
3/100mL           52
750mL + 3/        52
250mL             52
720mL             51
375mL 2 Pk        50
5.0 Oz            45
200mL 3 Pk        45
330mL             43
750mL + 2/        35
187mL 3 Pk        30
180mL             26
750mL 3 Pk        20
375mL 3 Pk        15
50mL 12 Pk         9
22.0 Oz            8
18L                6
6L                 1
20L                1
750mL + 4/         1
Name: count, dtype: int64

Valores únicos y sus conteos menos frecuentes (Bottom

In [5]:
# Reemplazar la entrada 'Liter' por '1 L' 
df['size'] = df['size'].str.replace('Liter', '1 L', regex=False)

# Confirma que el reemplazo se realizó correctamente
print("Valores después de corregir 'Liter':")
print(df[df['size'].str.contains('1 L', na=False)].head())

Valores después de corregir 'Liter':
          inventory_id  store          city  brand  \
8   1_HARDERSFIELD_115      1  HARDERSFIELD    115   
10  1_HARDERSFIELD_126      1  HARDERSFIELD    126   
19  1_HARDERSFIELD_254      1  HARDERSFIELD    254   
20  1_HARDERSFIELD_261      1  HARDERSFIELD    261   
25  1_HARDERSFIELD_294      1  HARDERSFIELD    294   

                    description size  on_hand  price  start_date  
8               Belvedere Vodka  1 L        5  27.99  2016-01-01  
10             Grey Goose Vodka  1 L       17  29.99  2016-01-01  
19  DeKuyper Root Beer Schnapps  1 L       13  10.99  2016-01-01  
20          Svedka Citron Vodka  1 L       14  12.99  2016-01-01  
25      Old Mexico Gold Tequila  1 L       28   8.49  2016-01-01  


In [6]:
# Filtra todas las filas que contienen la etiqueta ' Pk' (Paquete)
df_paquetes = df[df['size'].str.contains('Pk', na=False)]

print("Muestras de entradas con ' Pk' (Paquete):")
print(df_paquetes[['brand', 'description', 'size', 'on_hand']].head(10))

Muestras de entradas con ' Pk' (Paquete):
     brand                   description        size  on_hand
116   1001           Baileys 50mL 4 Pack   50mL 4 Pk        0
118   1005       Maker's Mark Combo Pack  375mL 2 Pk        7
119   1006   Jim Beam Candy Cane 4/50mLs   50mL 4 Pk        1
120   1009       Rebel Yell Variety Pack  750mL 3 Pk        1
131   1031  Avion Tasting Flight 3/375mL  375mL 3 Pk        5
135   1036          Jameson Trilogy Pack  200mL 3 Pk        5
144   1065  DiSaronno Cavalli Collection   50mL 3 Pk        5
149   1133  Glenmorangie Collection 4 Pk  100mL 4 Pk        0
171   1259  St George Gin 3/200mls Combo  200mL 3 Pk        1
196   1365   Stoli Mini Bar 5/50mLs Pack   50mL 5 Pk       49


In [7]:
# Eliminamos cualquier patrón que sea: [espacio opcional] + [número] + [espacio opcional] + [Pk o pK]
# Esto elimina ' 4 Pk', ' 2 Pk', ' 5 Pk', etc.
df['size'] = df['size'].str.replace(r'\s*\d+\s*[Pp][Kk]', '', regex=True)

# Verificamos los cambios
print("Valores de 'size' después de eliminar etiquetas de Paquete (Muestras):")
print(df[df['size'].str.contains('mL', na=False)]['size'].value_counts().head(5))

Valores de 'size' después de eliminar etiquetas de Paquete (Muestras):
size
750mL    144656
50mL      10042
375mL      7478
187mL       832
500mL       425
Name: count, dtype: int64


In [8]:
# Filtra las filas que tienen el valor ambiguo '3/100mL'
df_anomalia = df[df['size'].str.contains('3/100mL', na=False, regex=False)]

print("Registros a Investigar:")
# Muestra la Marca (Brand) y la Descripción (Description) de esas filas
print(df_anomalia[['brand', 'description', 'size','on_hand']].head(5))

# Ejemplo de búsqueda en Google (Usa los datos de tu output):
# Búsqueda: "[Brand] [Description] 3/100mL volumen"

Registros a Investigar:
       brand       description     size  on_hand
1338    5629  RumChata Liqueur  3/100mL       46
4483    5629  RumChata Liqueur  3/100mL       60
7660    5629  RumChata Liqueur  3/100mL       40
9970    5629  RumChata Liqueur  3/100mL        7
15294   5629  RumChata Liqueur  3/100mL       32


In [9]:
# Corrección basada en la investigación: El volumen unitario real es 100mL.
df['size'] = df['size'].str.replace('3/100mL', '100mL', regex=False)

In [10]:
#  Limpiar el formato '750mL + 3/' eliminando el texto residual
df['size'] = df['size'].str.replace(r'\s*\+\s*\d*/', '', regex=True)

In [11]:
# Definir los factores de conversión a ML (la unidad base)
factores_conversion = {
    'mL': 1.0,
    'L': 1000.0,
    'Oz': 29.5735
}

# 1. Extracción y Preparación
# Extrae el valor numérico y la unidad de la columna 'size' limpia
df[['Valor_temp', 'Unidad_temp']] = df['size'].str.extract(r'(\d+\.?\d*)\s*([a-zA-Z]+)', expand=True)

# Convierte el valor a float y mapea la unidad al factor de conversión
df['Valor_temp'] = pd.to_numeric(df['Valor_temp'], errors='coerce')
df['Factor'] = df['Unidad_temp'].map(factores_conversion)

# 2. CONVERSIÓN FINAL Y CREACIÓN DE LA COLUMNA (size_mL)
# Multiplica el valor por el factor (ej. 1.75 * 1000)
df['size_mL'] = df['Valor_temp'] * df['Factor']

# 3. Limpieza y Eliminación de la columna original
# Elimina la columna original 'size' y las columnas temporales.
df.drop(columns=['size', 'Valor_temp', 'Unidad_temp', 'Factor'], inplace=True)

# Revisa el resultado
print("\n--- ¡Columna size_mL Creada! ---")
print(f"Tipo de dato: {df['size_mL'].dtype}")
print(df[['inventory_id', 'size_mL']].head())


--- ¡Columna size_mL Creada! ---
Tipo de dato: float64
        inventory_id  size_mL
0  1_HARDERSFIELD_58    750.0
1  1_HARDERSFIELD_60    750.0
2  1_HARDERSFIELD_62    750.0
3  1_HARDERSFIELD_63    750.0
4  1_HARDERSFIELD_72    750.0


In [12]:
# Estandarización de la columna 'description' a mayúsculas y limpieza de espacios y puntuación

# 1. Convertir a mayúsculas.
# 2. Eliminar espacios iniciales y finales (str.strip()).
# 3. Eliminar cualquier carácter de puntuación o guion que pueda causar inconsistencia.
# 4. Reemplazar múltiples espacios internos (ej. '  ') por un solo espacio.

df['description'] = (
    df['description']
    .str.upper() # Todo a MAYÚSCULAS
    .str.strip() # Elimina espacios al inicio y al final
    .str.replace(r'[.,\-&/]', '', regex=True) # Elimina puntos, comas, guiones, etc.
    .str.replace(r'\s+', ' ', regex=True) # Reemplaza múltiples espacios por uno solo
)

print("La columna 'description' ha sido estandarizada a mayúsculas y se han limpiado los espacios y caracteres de puntuación.")

# Muestra un ejemplo de las primeras 5 descripciones limpias
print("\nEjemplo de descripciones estandarizadas:")
print(df['description'].head())

La columna 'description' ha sido estandarizada a mayúsculas y se han limpiado los espacios y caracteres de puntuación.

Ejemplo de descripciones estandarizadas:
0     GEKKEIKAN BLACK GOLD SAKE
1        CANADIAN CLUB 1858 VAP
2      HERRADURA SILVER TEQUILA
3    HERRADURA REPOSADO TEQUILA
4           NO 3 LONDON DRY GIN
Name: description, dtype: object


In [13]:
# Creación de la columna 'inventory_value' como el producto de 'on_hand' y 'price'

# 1. Asegurar que las columnas son numéricas (crítico para la multiplicación)
# Usamos 'float' para manejar posibles decimales en el precio o la cantidad
df['on_hand'] = pd.to_numeric(df['on_hand'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')


# 2. Calcular el valor total del inventario para cada fila (producto en tienda)
df['inventory_value'] = df['on_hand'] * df['price']

print("Columna 'inventory_value' creada exitosamente.")
print("\nEjemplo de las primeras 5 filas con el nuevo valor:")
print(df[['on_hand', 'price', 'inventory_value']].head())

Columna 'inventory_value' creada exitosamente.

Ejemplo de las primeras 5 filas con el nuevo valor:
   on_hand  price  inventory_value
0        8  12.99           103.92
1        7  10.99            76.93
2        6  36.99           221.94
3        3  38.99           116.97
4        6  34.99           209.94


## Establecemos los tipos de datos correctos

In [ ]:
# Convertir las claves a los tipos de datos correctos
df['brand'] = pd.to_numeric(df['brand'], errors='coerce').astype('Int64')
df['store'] = pd.to_numeric(df['store'], errors='coerce').astype('Int64')

# Usamos el tipo StringDtype de Pandas para una mejor gestión de cadenas.
df['description'] = df['description'].astype(str).astype(pd.StringDtype())
df['city'] = df['city'].astype(str).astype(pd.StringDtype())
df['inventory_id'] = df ['inventory_id'].astype(str).astype(pd.StringDtype())

In [16]:
# Convertir las métricas a tipos numéricos para cálculos
df['on_hand'] = pd.to_numeric(df['on_hand'], errors='coerce').astype('float64')
df['price'] = pd.to_numeric(df['price'], errors='coerce').astype('float64')
df['size_mL'] = pd.to_numeric(df['size_mL'], errors='coerce').astype('float64')
df['inventory_value'] = pd.to_numeric(df['inventory_value'], errors='coerce').astype('float64')

In [19]:
print("\nTipos de datos finales en BegInvFINAL:")
print(df.info())


Tipos de datos finales en BegInvFINAL:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   inventory_id     206529 non-null  string 
 1   store            206529 non-null  Int64  
 2   city             206529 non-null  string 
 3   brand            206529 non-null  Int64  
 4   description      206529 non-null  string 
 5   on_hand          206529 non-null  float64
 6   price            206529 non-null  float64
 7   start_date       206529 non-null  object 
 8   size_mL          206529 non-null  float64
 9   inventory_value  206529 non-null  float64
dtypes: Int64(2), float64(4), object(1), string(3)
memory usage: 16.2+ MB
None


In [20]:
# Convertir la columna 'start_date' al tipo de dato datetime.
# 'errors='coerce'' convertirá cualquier valor que no sea una fecha válida a NaT (Not a Time),
# lo cual es el valor nulo estándar para fechas.
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

print("La columna 'start_date' ha sido convertida a formato datetime.")

# Verificar el tipo de dato y las primeras filas
print("\nVerificación de formato:")
print(df['start_date'].head())
print(df['start_date'].dtype)

La columna 'start_date' ha sido convertida a formato datetime.

Verificación de formato:
0   2016-01-01
1   2016-01-01
2   2016-01-01
3   2016-01-01
4   2016-01-01
Name: start_date, dtype: datetime64[ns]
datetime64[ns]


## Guardamos el archivo limpio

In [22]:
import os
# --- Definir el Nombre del Archivo de Salida ---
nombre_archivo_salida = 'BegInvFINAL_LIMPIO.csv'

# --- Exportar a CSV ---
df.to_csv(
    nombre_archivo_salida, 
    index=False,             # CRÍTICO: No incluir el índice de Pandas (evita columnas innecesarias)
    encoding='utf-8'         # CRÍTICO: Usar codificación UTF-8 para manejar caracteres especiales
)

# --- Verificar la Ubicación ---
# Mostrar la ruta completa donde se guardó el archivo
ruta_completa = os.path.abspath(nombre_archivo_salida)

print(f"✅ ¡ETL completado! El archivo CSV ha sido guardado.")
print(f"Nombre del archivo: {nombre_archivo_salida}")
print(f"Ubicación completa: {ruta_completa}")

✅ ¡ETL completado! El archivo CSV ha sido guardado.
Nombre del archivo: BegInvFINAL_LIMPIO.csv
Ubicación completa: c:\Users\USUARIO\Downloads\archive\BegInvFINAL_LIMPIO.csv
